Lower-bounding f_n and h_n using a mixed integer programming framework (with the Branch-and-cut algorithm).

In [1]:
%run constants.ipynb
%run gen_h.ipynb
import gurobipy as gp
from gurobipy import GRB
from itertools import repeat
import re

In [2]:
def var_name(*ineqs, prefix=''):
    '''
    Returns the variable name for the MIP.
    '''
    return prefix + f'{str(ineqs)}'

In [3]:
def extract_ineq(var):
    '''
    Extracts the inequality from the given variable's name.
    (For interpretability, we associate each indicator variable's name 
    with the inequality(s) they represent)
    '''
    match = re.findall(r'\(([^()]+)\)', var.varName)
    return sympify(match[0]) if match else None

In [4]:
def extract_ineqs(variables):
    '''
    Extracts inequalities from multiple variables, 
    based on their values (i.e. 1 = true, 0 = false).
    '''
    ineqs = []
    for v in variables:
        e = extract_ineq(v)
        if v.x == 1 and e is not None:  # indicator is true
            ineqs.append(e)
        elif v.x == 0:  # indicator is false
            ineqs.append(Not(And(e)))
    return ineqs

In [5]:
def to_gp_linexpr(sp_expr, var_map):
    '''
    Converts a sympy expression to a gurobipy linear expression,
    using the provided (sp : gp) variable mapping.
    '''
    gp_expr = gp.LinExpr()

    if isinstance(sp_expr, Add):
        for term in sp_expr.args:
            gp_expr.add(to_gp_linexpr(term, var_map))
    elif sp_expr.is_Mul:
        coeff = 1
        var = None
        for factor in sp_expr.args:
            if factor.is_number:  # sp coefficient
                coeff *= float(factor)
            elif factor in var_map:  # sp variable
                var = var_map[factor]
        if var is not None:
            gp_expr.addTerms(coeff, var)
        else:  # constant
            gp_expr.addConstant(coeff)
    elif sp_expr in var_map:  # single variable expression
        gp_expr.addTerms(1.0, var_map[sp_expr])
    elif sp_expr.is_number:   # single constant
        gp_expr.addConstant(float(sp_expr))

    return gp_expr

In [6]:
EPS = 1
def unify_ineq(ineq, eps, scale_by=1):
    '''
    Converts the given inequality to the form expr >= 0.
    '''
    lhs, rhs, op = ineq.lhs, ineq.rhs, ineq.rel_op
    # Check the inequality type and unify the format
    if op == '>=':
        # lhs >= rhs -> lhs - rhs >= 0
        return Rel(scale_by * (lhs - rhs), 0, '>=')
    elif op == '>':
        # lhs > rhs -> lhs - rhs > 0 => lhs - rhs >= eps
        return Rel(scale_by * (lhs - rhs) - eps, 0, '>=')
    elif op == '<=':
        # lhs <= rhs -> lhs - rhs <= 0 -> rhs - lhs >= 0
        return Rel(scale_by * (rhs - lhs), 0, '>=')
    elif op == '<':
        # lhs < rhs -> rhs > lhs -> rhs - lhs > 0 -> rhs - lhs >= eps
        return Rel(scale_by * (rhs - lhs - eps), 0, '>=')
    else:
        raise ValueError(f"Unhandled inequality type: {op}")

In [7]:
def to_gp_ineq(ineq, var_map, scale_by=1, eps=EPS):
    '''
    Like `to_gp_linexpr`, but converts a sympy inequality to a gurobipy inequality 
    (of the form >= 0).
    '''
    return (to_gp_linexpr(unify_ineq(ineq, eps, scale_by=scale_by).lhs, var_map) >= 0)

In [8]:
def add_ind_constr(ineq, model=None, name='ind', var_map=None, eps=EPS, scale_by=1):
    ''' 
    Adds indicator constraint to the model for a single inequality. 
    '''
    lhs = to_gp_linexpr(unify_ineq(ineq, eps, scale_by=scale_by).lhs, var_map)  # to the form >= 0
    v = model.addVar(vtype=GRB.BINARY, name=name)
    model.addConstr((v == 1) >> (lhs >= 0))
    model.addConstr((v == 0) >> (lhs <= -eps))  # lhs < 0
    return v

In [9]:
def add_ind_constrs(*ineqs, model=None, var_map=None, eps=EPS, scale_by=1):
    ''' 
    Adds variables and constraints to the model for a compound indicator 
    (i.e. may represent the and of multiple inequalities). 
    '''
    if len(ineqs) == 1:
        return (add_ind_constr(ineqs[0], 
                               name=var_name(ineqs[0], prefix='single_ind'), 
                               var_map=var_map,
                               model=model,
                               scale_by=scale_by,
                               eps=eps),)
    
    I = model.addVar(vtype=GRB.BINARY, 
                     name=var_name(*ineqs, prefix='main_ind'))
    component_vars = [add_ind_constr(i, 
                                     name=var_name(i, prefix='component_ind'), 
                                     var_map=var_map,
                                     model=model,
                                     scale_by=scale_by,
                                     eps=eps) for i in ineqs]
    model.addConstr(I == gp.and_(component_vars), 
                    name=var_name(*ineqs, prefix='andconstr'))
    return I, component_vars

In [10]:
def analyse_result(model, relax=False):
    '''
    Analyses the status of the model.
    If the model is feasible and `relax` is True, then re-optimizes the model with relaxed bounds.
    Note that if the relaxation is infeasible, then the exact model must too be infeasible
    '''
    if model.status == GRB.OPTIMAL:
        print('Optimal solution found:')
        for v in model.getVars():
            print(f"{v.varName}: {v.x}")
        print(f'Objective value:', model.objVal)
        print(f'MIPGap = |ObjBound - ObjVal|/|ObjVal|:', model.MIPGap)
    elif model.status == GRB.UNBOUNDED:
        print('The model cannot be solved because it is unbounded')
    elif model.status != GRB.INF_OR_UNBD and model.status != GRB.INFEASIBLE:
        print('Optimization was stopped with status %d' % model.status)
    else:
        # Relax the constraints and try to make the model feasible
        print('The model is infeasible')
        if relax:
            print('Relaxing the bounds')
            model.feasRelaxS(1, False, False, True)
            model.optimize()
            analyse_result(model, relax=relax)

In [15]:
def delta_mip(n, output_file=False, analyse=True, relax=False):
    '''
    Runs MIP model to minimize f_n - thresh(n).
    If `output_file` is True, writes the model as a '.lp' file.
    If `analyse` is True, analyses the optimization results.
    '''
    # uncomment the following to collect the regions cut out (i.e. excluded) by the model
#     failed_regions = []
    
#     def extract_region(model):
#         region = []
#         variables = model.getVars()
#         sol = model.cbGetSolution(variables)
#         for v, val in zip(variables, sol):
#             # only look at the individual indicators
#             if not 'single_ind' in v.varName and not 'component_ind' in v.varName:
#                 continue
#             e = extract_ineq(v)
#             if val == 1 and e is not None:
#                 region.append(e[0])
#             elif val == 0:
#                 # indicator is false
#                 # guaranteed single inequality
#                 region.append(Not(e[0]))
#         return region
    
#     def pos_f_callback(model, where):
#         if where == GRB.Callback.MIPSOL:
#             # Capture node information
#             try:
#                 obj = model.cbGet(GRB.Callback.MIPSOL_OBJ)
#                 print("Current objective:", obj)
#                 if obj > 0:  # Node is infeasible
#                     failed_regions.append(extract_region(model))
#             except gp.GurobiError as e:
#                 print(f"Gurobi error during callback: {e}")
#             except Exception as e:
#                 print(f"Unexpected error during callback: {e}")

    def bnd_callback(model, where):
        if where == gp.GRB.Callback.MIP:
            cur_obj = model.cbGet(gp.GRB.Callback.MIP_OBJBST)
            cur_bnd = model.cbGet(gp.GRB.Callback.MIP_OBJBND)
            
            if model._obj != cur_obj or model._bnd != cur_bnd:
                model._obj = cur_obj
                model._bnd = cur_bnd
                print("New objective:", cur_obj)
                print("New bound:", cur_bnd)
            
    
    pos_h = read(POS_H_CACHE, min_n=1, max_n=n)
    neg_h = read(NEG_H_CACHE, min_n=1, max_n=n)
    num_pos_per_n = [len(pos_h[i][1]) for i in range(1,n+1)]
    num_neg_per_n = [len(neg_h[i][1]) for i in range(1,n+1)]
    num_pos = sum(num_pos_per_n)
    num_neg = sum(num_neg_per_n)
    
    pos_coefs = concat([repeat(j, k) for j,k in zip([coef(i) for i in range(1,n+1)], num_pos_per_n)])
    neg_coefs = concat([repeat(j, k) for j,k in zip([-coef(i) for i in range(1,n+1)], num_neg_per_n)])
    const = sum([coef(i)*(pos_h[i][0] - neg_h[i][0]) for i in range(1,n+1)])
    
    all_pos_comp_inds = concat([pos_h[i][1] for i in range(1,n+1)])
    all_neg_comp_inds = concat([neg_h[i][1] for i in range(1,n+1)])
    
    model = gp.Model(F_MODEL_NAME(n), env=gp.Env())
    model.Params.OutputFlag = 1
    model.Params.LogToConsole = 1
    # nodes in the B&B tree are compressed and written to disk - reduces memory use
    model.Params.NodefileStart = 0.5
#     model.Params.PreSparsify = 1
    model.Params.MIPFocus = 1
    
    model._obj = None
    model._bnd = None
    
    gx = model.addVar(lb=0, name='x')
    gy = model.addVar(lb=0, name='y')
    gz = model.addVar(lb=0, name='z')
    sp_to_gp = {x: gx, y: gy, z: gz}
    
    pos_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp, eps=EPS) \
                for i in all_pos_comp_inds]
    neg_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp, eps=EPS) \
                for i in all_neg_comp_inds]

    obj = gp.quicksum([pos_coefs[i]*pos_inds[i][0] for i in range(num_pos)]) + \
        gp.quicksum([neg_coefs[i]*neg_inds[i][0] for i in range(num_neg)]) + const - thresh(n)
    
    # constraints for V
    for i, ineq in enumerate(V):
        model.addConstr(to_gp_ineq(ineq, sp_to_gp, eps=EPS), name=f'V{i}')
        
    delta_constr = model.addConstr(obj <= 0, name='non_pos_delta')
    
#     model.optimize(pos_f_callback)
    model.setObjective(obj, GRB.MINIMIZE)
#     model.setObjective(0, GRB.MAXIMIZE)
#     model.feasRelaxS(1, False, False, True)
    model.optimize(callback=bnd_callback)
    
    if analyse:
        analyse_result(model, relax=relax)
    
    if output_file:
        model.write(F_MIP_OUTPUT(n))
    
    return ((model.status == GRB.OPTIMAL and \
            model.MIPGap == 0 and \
            sub(V, {x:gx.x, y:gy.x, z:gz.x}) == 1 and \
            eval_delta(n, gx.x, gy.x, gz.x) < 0), # sanity check using the x,y,z values from the model
            model)

In [16]:
status, model = delta_mip(5, analyse=True, relax=False)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Set parameter NodefileStart to value 0.5
Set parameter MIPFocus to value 1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 10690 columns and 3097 nonzeros
Model fingerprint: 0x5b053e66
Model has 18566 general constraints
Variable types: 3 continuous, 10687 integer (10687 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  Objective range  [1e-04, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-02, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 3e+01]
Presolve added 27586 rows and 23637 columns
Presolve time: 1.43s
Presolved: 27590 rows, 34327 columns, 84178 nonzeros
Presolved model has 15758 SOS constraint(s)
Variable types: 15761 continuous, 18566 integer (18566 binary)
New obj

 115279  5428   -0.52050 2210 1815   -0.02320   -0.70954  2958%  17.8  280s
 115823  6420   -0.52050 2278 1746   -0.02320   -0.70954  2958%  17.7  287s
 116447  7050   -0.42251 2356 1689   -0.02320   -0.70954  2958%  17.6  291s
 117077  7242   -0.41415 2410 1659   -0.02320   -0.70954  2958%  17.5  296s
 117525  8106   -0.41016 2466 1624   -0.02320   -0.70954  2958%  17.5  302s
 118133  8650   -0.41016 2542 1549   -0.02320   -0.70954  2958%  17.4  307s
 118938  9431   -0.41016 2666 1422   -0.02320   -0.70954  2958%  17.2  313s
 119458  9565   -0.39100 2729 1378   -0.02320   -0.70954  2958%  17.2  325s
 119905 10646   -0.38701 2780 1341   -0.02320   -0.70954  2958%  17.1  333s
 120673 11242   -0.21366 2876 1246   -0.02320   -0.70954  2958%  17.0  337s
 121269 12058   -0.21366 2904 1221   -0.02320   -0.70954  2958%  16.9  342s
 122085 12979   -0.21366 3036 1088   -0.02320   -0.70954  2958%  16.8  348s
 123006 14266   -0.18549 3152  969   -0.02320   -0.70954  2958%  16.7  353s
 124293 1513

 213560 57370   -0.35782 3171  918   -0.02487   -0.59059  2275%  15.2  866s
 214375 57687   -0.35782 3192  897   -0.02487   -0.59059  2275%  15.2  873s
 215342 58144   -0.35730 3326  777   -0.02487   -0.59059  2275%  15.1  876s
 217245 59066 infeasible 3494        -0.02487   -0.59059  2275%  15.0  883s
 217873 59627 infeasible 3522        -0.02487   -0.59059  2275%  15.2  887s
 218960 60182   -0.34920 3552  560   -0.02487   -0.59059  2275%  15.4  896s
 219957 60770 infeasible 3580        -0.02487   -0.59059  2275%  15.4  901s
 221213 61127   -0.34907 3604  508   -0.02487   -0.59059  2275%  15.3  905s
 222216 61537 infeasible 3644        -0.02487   -0.59059  2275%  15.3  910s
 224204 61987 infeasible 3704        -0.02487   -0.59059  2275%  15.2  917s
 225620 62391   -0.17058 3856  297   -0.02487   -0.59059  2275%  15.2  920s
 226618 62855 infeasible 3896        -0.02487   -0.59059  2275%  15.1  928s
 227718 63390   -0.07644 3938  242   -0.02487   -0.59059  2275%  15.1  932s
 228893 6369

 331555 98173   -0.11451 3406  766   -0.02487   -0.52924  2028%  16.0 1406s
 333166 98762 infeasible 3510        -0.02487   -0.52924  2028%  16.0 1412s
 333669 99187   -0.08493 3543  671   -0.02487   -0.52924  2028%  16.0 1417s
 334270 99740 infeasible 3599        -0.02487   -0.52924  2028%  16.0 1421s
 335047 100397   -0.05136 3664  553   -0.02487   -0.52924  2028%  15.9 1429s
 336312 101072   -0.05085 3727  495   -0.02487   -0.52924  2028%  16.0 1433s
 337561 101239 infeasible 3783        -0.02487   -0.52924  2028%  16.0 1437s
 338592 101475   -0.03529 3835  419   -0.02487   -0.52680  2018%  16.0 1440s
New objective: -0.024871399176954828
New bound: -0.5268004115226338
 339258 101811   -0.51986 2173 2010   -0.02487   -0.52680  2018%  16.0 1449s
 340194 102145   -0.51986 2226 1946   -0.02487   -0.52680  2018%  16.0 1452s
 341176 102499   -0.42958 2313 1874   -0.02487   -0.52680  2018%  16.0 1459s
 341926 102888   -0.41132 2391 1831   -0.02487   -0.52680  2018%  16.0 1464s
 342893 1033

 450350 140991   -0.41235 2400 1575   -0.02487   -0.51214  1959%  16.6 1962s
 451296 141540   -0.41132 2472 1513   -0.02487   -0.51214  1959%  16.6 1966s
 452141 142036   -0.41106 2533 1453   -0.02487   -0.51214  1959%  16.6 1970s
 453390 143346   -0.41106 2681 1306   -0.02487   -0.51214  1959%  16.5 1979s
 454279 144138   -0.41106 2786 1200   -0.02487   -0.51214  1959%  16.5 1984s
 455399 144839   -0.40772 2900 1097   -0.02487   -0.51214  1959%  16.5 1989s
 456160 145759   -0.39524 3002 1011   -0.02487   -0.51214  1959%  16.4 1994s
 457410 146423   -0.27976 3131  934   -0.02487   -0.51214  1959%  16.4 1999s
 458164 147332   -0.27294 3222  886   -0.02487   -0.51214  1959%  16.4 2004s
 459417 148125   -0.26870 3359  771   -0.02487   -0.51214  1959%  16.3 2010s
 462197 148480   -0.09380 3660  489   -0.02487   -0.51214  1959%  16.3 2017s
 463323 149162   -0.09187 3764  392   -0.02487   -0.51214  1959%  16.3 2020s
New objective: -0.024871399176954828
New bound: -0.5107253086419753
 465441 

...

 9655347 29938 infeasible 3966        -0.02487   -0.02770  11.4%  20.0 64513s
New objective: -0.024871399176954828
New bound: -0.02770061728395056
New objective: -0.024871399176954828
New bound: -0.027700617283950537
New objective: -0.024871399176954828
New bound: -0.027700617283950534
New objective: -0.024871399176954828
New bound: -0.02770061728395052
New objective: -0.024871399176954828
New bound: -0.027700617283950516
New objective: -0.024871399176954828
New bound: -0.027700617283950506
New objective: -0.024871399176954828
New bound: -0.027700617283950475
New objective: -0.024871399176954828
New bound: -0.027700617283950464
New objective: -0.024871399176954828
New bound: -0.027572016460905353
 9656117 29046 infeasible 3975        -0.02487   -0.02757  10.9%  20.0 64518s
New objective: -0.024871399176954828
New bound: -0.027572016460905308
New objective: -0.024871399176954828
New bound: -0.027572016460905305
New objective: -0.024871399176954828
New bound: -0.02757201646090529
New obj

 9678473 15224 infeasible 4035        -0.02487   -0.02680  7.76%  20.0 64570s
New objective: -0.024871399176954828
New bound: -0.02680041152263374
New objective: -0.024871399176954828
New bound: -0.026800411522633735
New objective: -0.024871399176954828
New bound: -0.026800411522633724
New objective: -0.024871399176954828
New bound: -0.02680041152263371
New objective: -0.024871399176954828
New bound: -0.026800411522633707
New objective: -0.024871399176954828
New bound: -0.026800411522633696
New objective: -0.024871399176954828
New bound: -0.026800411522633672
New objective: -0.024871399176954828
New bound: -0.026800411522633658
New objective: -0.024871399176954828
New bound: -0.02680041152263364
New objective: -0.024871399176954828
New bound: -0.026800411522633637
New objective: -0.024871399176954828
New bound: -0.026800411522633634
New objective: -0.024871399176954828
New bound: -0.02680041152263363
New objective: -0.024871399176954828
New bound: -0.026800411522633627
New objective: -

 9689336  6796 infeasible 3999        -0.02487   -0.02616  5.17%  20.0 64597s
New objective: -0.024871399176954828
New bound: -0.026157407407407407
New objective: -0.024871399176954828
New bound: -0.026157407407407404
New objective: -0.024871399176954828
New bound: -0.02615740740740739
New objective: -0.024871399176954828
New bound: -0.02615740740740737
New objective: -0.024871399176954828
New bound: -0.026157407407407355
New objective: -0.024871399176954828
New bound: -0.02615740740740735
New objective: -0.024871399176954828
New bound: -0.026157407407407327
New objective: -0.024871399176954828
New bound: -0.026157407407407313
New objective: -0.024871399176954828
New bound: -0.026157407407407306
New objective: -0.024871399176954828
New bound: -0.02615740740740729
New objective: -0.024871399176954828
New bound: -0.026157407407407286
New objective: -0.024871399176954828
New bound: -0.026157407407407282
New objective: -0.024871399176954828
New bound: -0.02615740740740728
New objective: -0

New objective: -0.024871399176954828
New bound: -0.025514403292180982
New objective: -0.024871399176954828
New bound: -0.025514403292180927
New objective: -0.024871399176954828
New bound: -0.025514403292180916
New objective: -0.024871399176954828
New bound: -0.02551440329218091
New objective: -0.024871399176954828
New bound: -0.025385802469135806
 9698342   907 infeasible 4053        -0.02487   -0.02539  2.07%  19.9 64615s
New objective: -0.024871399176954828
New bound: -0.025385802469135802
New objective: -0.024871399176954828
New bound: -0.025385802469135747
New objective: -0.024871399176954828
New bound: -0.025385802469135733
New objective: -0.024871399176954828
New bound: -0.02538580246913571
New objective: -0.024871399176954828
New bound: -0.025385802469135688
New objective: -0.024871399176954828
New bound: -0.02538580246913566
New objective: -0.024871399176954828
New bound: -0.025385802469135656
New objective: -0.024871399176954828
New bound: -0.025385802469135646
New objective: 

component_ind(4*y < 7*z,): 0.0
main_ind(8*y <= 13*z, 3*z < 2*y): 0.0
component_ind(8*y <= 13*z,): 0.0
component_ind(3*z < 2*y,): 1.0
main_ind(2*y <= 7*z, 7*z < 4*y, y < 2*z): 0.0
component_ind(2*y <= 7*z,): 0.0
component_ind(7*z < 4*y,): 1.0
component_ind(y < 2*z,): 0.0
main_ind(8*y <= 15*z, 7*z < 4*y): 0.0
component_ind(8*y <= 15*z,): 0.0
component_ind(7*z < 4*y,): 1.0
main_ind(z <= 2*y, 3*z < 2*y, z < y, y < 2*z): 0.0
component_ind(z <= 2*y,): 1.0
component_ind(3*z < 2*y,): 1.0
component_ind(z < y,): 1.0
component_ind(y < 2*z,): 0.0
main_ind(6*y <= 11*z, 3*z < 2*y, z < y): 0.0
component_ind(6*y <= 11*z,): 0.0
component_ind(3*z < 2*y,): 1.0
component_ind(z < y,): 1.0
main_ind(z < y, z < 2*y, y < 2*z): 0.0
component_ind(z < y,): 1.0
component_ind(z < 2*y,): 1.0
component_ind(y < 2*z,): 0.0
main_ind(2*y <= 3*z, z < y, z < 2*y): 0.0
component_ind(2*y <= 3*z,): 0.0
component_ind(z < y,): 1.0
component_ind(z < 2*y,): 1.0
main_ind(z < y, 2*z < 3*y, 3*y < 5*z): 0.0
component_ind(z < y,): 1.0

In [21]:
status, model = delta_mip(4, analyse=True, relax=False)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 1712 columns and 577 nonzeros
Model fingerprint: 0x6b7d2b75
Model has 2953 general constraints
Variable types: 3 continuous, 1709 integer (1709 binary)
Coefficient statistics:
  Matrix range     [8e-04, 1e+00]
  Objective range  [8e-04, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-02, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 2e+01]
Presolve added 4376 rows and 3732 columns
Presolve time: 0.06s
Presolved: 4380 rows, 5444 columns, 13077 nonzeros
Presolved model has 2488 SOS constraint(s)
Variable types: 2491 continuous, 2953 integer (2953 binary)

Root relaxation: objective -6.495370e-01, 2614 iterations, 0.01 seconds (0.01 work units)

Int

In [24]:
eval_delta(4, 
           model.getVarByName('x').x, 
           model.getVarByName('y').x, 
           model.getVarByName('z').x)

-0.05

In [21]:
eval_delta(5, 50,51,6)

-0.024871399176954734

# LB on h_n(x,y,z) - h_n(y,x,z)

In [17]:
def h_mip(n, output_file=False, analyse=True, relax=False):
    '''
    Like `f_mip`, but runs MIP model to minimize h_n(x,y,z) - h_n(y,x,z).
    '''
    pos_h = read(POS_H_CACHE, min_n=n, max_n=n)
    neg_h = read(NEG_H_CACHE, min_n=n, max_n=n)
    
    model = gp.Model(DH_MODEL_NAME(n), env=gp.Env())
    model.Params.OutputFlag = 1
    model.Params.LogToConsole = 1
#     model.Params.NodeLimit = 100000
    model.Params.MIPFocus = 3
    
    gx = model.addVar(lb=0, name='x')
    gy = model.addVar(lb=0, name='y')
    gz = model.addVar(lb=0, name='z')
    sp_to_gp = {x: gx, y: gy, z: gz}
    
    pos_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp, eps=EPS) for i in pos_h[n][1]]
    neg_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp, eps=EPS) for i in neg_h[n][1]]
    const = pos_h[n][0] - neg_h[n][0]
    
    obj = coef(n) * (gp.quicksum([i[0] for i in pos_inds]) - gp.quicksum([i[0] for i in neg_inds]) + const)
    
    model.setObjective(obj, GRB.MINIMIZE)
    for i, ineq in enumerate(V):
        model.addConstr(to_gp_ineq(ineq, sp_to_gp, eps=EPS), name=f'V{i}')
    
    model.optimize()
    
    if analyse:
        analyse_result(model, relax=relax)
    
    if output_file:
        model.write(DH_MIP_OUTPUT(n))
    
    return (model.status == GRB.OPTIMAL and model.MIPGap == 0), model

In [28]:
h_mip(5, analyse=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Set parameter MIPFocus to value 3
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 8981 columns and 4 nonzeros
Model fingerprint: 0x325d6f1a
Model has 15613 general constraints
Variable types: 3 continuous, 8978 integer (8978 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-04, 1e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 3e+01]
Presolve added 23208 rows and 19905 columns
Presolve time: 1.54s
Presolved: 23211 rows, 28886 columns, 68583 nonzeros
Presolved model has 13270 SOS constraint(s)
Variable types: 13273 continuous, 15613 integer (15613 binary)
Root relaxation presolve removed 23211 rows and 28886 

 386221 17768 infeasible 1234        -0.00219   -0.03305  1412%  24.9  435s
 391052 19064 infeasible 1552        -0.00219   -0.03279  1400%  24.8  440s
 397170 19415 infeasible 1183        -0.00219   -0.03266  1394%  24.5  445s
 402419 19484 infeasible 1546        -0.00219   -0.03241  1382%  24.4  451s
 407864 20400 infeasible 1524        -0.00219   -0.03228  1376%  24.2  455s
 413260 22058   -0.03215 1300 1009   -0.00219   -0.03215  1371%  24.0  461s
 418270 21977 infeasible 1546        -0.00219   -0.03202  1365%  23.9  465s
 425067 22445 infeasible 1320        -0.00219   -0.03176  1353%  23.7  471s
 429437 22966 infeasible 1135        -0.00219   -0.03164  1347%  23.7  475s
 436101 24963 infeasible 1446        -0.00219   -0.03151  1341%  23.5  480s
 440770 24691 infeasible 1538        -0.00219   -0.03125  1329%  23.4  485s
 446966 24590 infeasible 1242        -0.00219   -0.03112  1324%  23.3  490s
 452230 24933 infeasible 1104        -0.00219   -0.03099  1318%  23.2  495s
 456825 2529

 934151 39016 infeasible 1728        -0.00219   -0.02096   859%  17.7 1026s
 937196 38379   -0.02083 1835  576   -0.00219   -0.02083   853%  17.7 1030s
 943930 37686 infeasible 1842        -0.00219   -0.02083   853%  17.7 1035s
 948521 38526 infeasible 1867        -0.00219   -0.02070   847%  17.7 1040s
 953450 37863 infeasible 2032        -0.00219   -0.02058   841%  17.6 1047s
 959530 38180 infeasible 1931        -0.00219   -0.02058   841%  17.6 1051s
 963170 37789 infeasible 1886        -0.00219   -0.02045   835%  17.6 1056s
 969874 36620 infeasible 1549        -0.00219   -0.02045   835%  17.6 1060s
 974412 36040 infeasible 1943        -0.00219   -0.02032   829%  17.6 1065s
 978595 35523 infeasible 2002        -0.00219   -0.02019   824%  17.5 1070s
 985377 34760 infeasible 1911        -0.00219   -0.02019   824%  17.5 1075s
 989729 34767 infeasible 1743        -0.00219   -0.02006   818%  17.5 1081s
 994084 34657 infeasible 1931        -0.00219   -0.01993   812%  17.4 1086s
 1000125 351

 1494271 43868 infeasible 1936        -0.00219   -0.01440   559%  14.7 1561s
 1501477 43562 infeasible 1933        -0.00219   -0.01440   559%  14.7 1567s
 1507483 43368 infeasible 1746        -0.00219   -0.01440   559%  14.7 1571s
 1509648 43076 infeasible 2139        -0.00219   -0.01427   553%  14.7 1575s
 1517786 42693 infeasible 2217        -0.00219   -0.01427   553%  14.7 1582s
 1522762 43250 infeasible 1770        -0.00219   -0.01427   553%  14.7 1586s
 1526432 43121 infeasible 2058        -0.00219   -0.01415   547%  14.6 1592s
 1530612 42914 infeasible 2041        -0.00219   -0.01415   547%  14.6 1595s
 1537879 43303 infeasible 1887        -0.00219   -0.01415   547%  14.6 1600s
 1544224 43483 infeasible 2021        -0.00219   -0.01402   541%  14.6 1608s
 1547568 43975 infeasible 2167        -0.00219   -0.01402   541%  14.5 1612s
 1554268 44658 infeasible 2170        -0.00219   -0.01402   541%  14.5 1616s
 1558369 45410 infeasible 1994        -0.00219   -0.01389   535%  14.5 1622s

 2059738 41563 infeasible 2239        -0.00219   -0.01055   382%  13.1 2098s
 2063980 41645 infeasible 2056        -0.00219   -0.01055   382%  13.1 2102s
 2067916 41883 infeasible 2081        -0.00219   -0.01055   382%  13.1 2106s
 2072176 41748 infeasible 1986        -0.00219   -0.01055   382%  13.1 2110s
 2080621 41767 infeasible 2014        -0.00219   -0.01055   382%  13.1 2115s
 2082608 40886 infeasible 1951        -0.00219   -0.01042   376%  13.1 2120s
 2089693 39896 infeasible 2253        -0.00219   -0.01042   376%  13.0 2127s
 2093939 39644 infeasible 2236        -0.00219   -0.01042   376%  13.0 2131s
 2100375 39072 infeasible 2112        -0.00219   -0.01042   376%  13.0 2135s
 2105876 39628 infeasible 2312        -0.00219   -0.01029   371%  13.0 2143s
 2109663 39946 infeasible 1945        -0.00219   -0.01029   371%  13.0 2147s
 2114175 39900 infeasible 2052        -0.00219   -0.01029   371%  13.0 2151s
 2117947 40058 infeasible 1964        -0.00219   -0.01029   371%  13.0 2155s

 2611539 32334 infeasible 2145        -0.00219   -0.00772   253%  11.8 2633s
 2615597 32469 infeasible 2300        -0.00219   -0.00772   253%  11.8 2637s
 2619492 32222 infeasible 2288        -0.00219   -0.00772   253%  11.8 2640s
 2626583 32188   -0.00772 2161  395   -0.00219   -0.00772   253%  11.8 2646s
 2629039 32246 infeasible 2303        -0.00219   -0.00759   247%  11.8 2651s
 2633097 32107 infeasible 2055        -0.00219   -0.00759   247%  11.7 2658s
 2637206 31850 infeasible 1989        -0.00219   -0.00759   247%  11.7 2664s
 2641351 31567 infeasible 2051        -0.00219   -0.00759   247%  11.7 2668s
 2644832 31566 infeasible 2083        -0.00219   -0.00759   247%  11.7 2675s
 2648849 31791 infeasible 2253        -0.00219   -0.00759   247%  11.7 2682s
 2651610 31720 infeasible 2126        -0.00219   -0.00759   247%  11.7 2686s
 2655964 31267 infeasible 2100        -0.00219   -0.00746   241%  11.7 2695s
 2659770 31088 infeasible 2059        -0.00219   -0.00746   241%  11.7 2701s

(True,
 <gurobi.Model MIP instance h5(x,y,z)-h5(y,y,z): 3 constrs, 8981 vars, Parameter changes: MIPFocus=3, Username=(user-defined)>)

In [29]:
eval_dh(5,16,17,16)

Fraction(-41, 7776)

In [33]:
status, model = h_mip(6, analyse=True)

KeyError: 6

# f(x, x, z) - f((x+y)/2, (x+y)/2, z) where x+y+z = FIXED_SUM

In [18]:
n=6
neg_h = read(NEG_H_CACHE, min_n=n, max_n=n)

In [13]:
num_pos_per_n = [len(pos_h[i][1]) for i in range(1,n+1)]
num_neg_per_n = [len(neg_h[i][1]) for i in range(1,n+1)]
num_pos = sum(num_pos_per_n)
num_neg = sum(num_neg_per_n)

pos_coefs = concat([repeat(j, k) for j,k in zip([coef(i) for i in range(1,n+1)], num_pos_per_n)])
neg_coefs = concat([repeat(j, k) for j,k in zip([-coef(i) for i in range(1,n+1)], num_neg_per_n)])
const = sum([coef(i)*(neg_h[i][0] - neg_h[i][0]) for i in range(1,n+1)])

/var/folders/tk/2zq7clf94gx3rdh5slf6rhc00000gn/T/ipykernel_1082/3349906455.py:14: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if not isinstance(args, collections.Hashable):


In [14]:
all_pos_comp_inds = concat([pos_h[i][1] for i in range(1,n+1)])
all_neg_comp_inds = concat([neg_h[i][1] for i in range(1,n+1)])

## Real version

In [48]:
EPS = 1
SCALE=10000

In [49]:
model = gp.Model(f"dh'{n}_model", env=gp.Env())
model.Params.OutputFlag = 1
model.Params.LogToConsole = 1

gx = model.addVar(lb=EPS, ub=SCALE, name='x')
gy = model.addVar(lb=EPS, ub=SCALE, name='y')
gz = model.addVar(lb=Z_LB * SCALE, ub=SCALE, name='z')
# dummy variable for the x in h(x,x,z), since this x'=(1-z)/2=(x+y)/2
sp_to_gp = {x: gx, y: gy, z: gz}

# constraints for V
model.addConstr(gx + gy + gz == SCALE, name=f'x+y+z=1')
model.addConstr(gx <= gy - EPS, name=f'x<y')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Set parameter MIPFocus to value 1


<gurobi.Constr *Awaiting Model Update*>

In [50]:
pos_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp, eps=EPS) \
                for i in all_pos_comp_inds]
neg_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp, eps=EPS) \
            for i in all_neg_comp_inds]

In [51]:
obj = gp.quicksum([pos_coefs[i]*pos_inds[i][0] for i in range(num_pos)]) + \
        gp.quicksum([neg_coefs[i]*neg_inds[i][0] for i in range(num_neg)]) + const - thresh(n)
    
model.setObjective(obj, GRB.MINIMIZE)

In [52]:
delta_constr = model.addConstr(obj <= 0, name='non_pos_delta')

In [53]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 5856 columns and 1895 nonzeros
Model fingerprint: 0x9a011c5a
Model has 10156 general constraints
Variable types: 0 continuous, 5856 integer (5853 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  Objective range  [1e-04, 3e-02]
  Bounds range     [1e+00, 1e+04]
  RHS range        [3e-02, 1e+04]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 3e+01]
Presolve added 10440 rows and 0 columns
Presolve removed 0 rows and 87 columns
Presolve time: 0.48s
Presolved: 10443 rows, 5769 columns, 39265 nonzeros
Variable types: 0 continuous, 5769 integer (5766 binary)

Root relaxation: objective -1.976494e-01, 6407 iterations, 0.59 seconds (1.35 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth

In [42]:
analyse_result(model)

Optimal solution found:
x: 308.0
y: 346.0
z: 346.0
single_ind(3*x <= y,): 0.0
single_ind(2*x <= z,): 0.0
single_ind(2*x <= y,): 0.0
single_ind(3*x <= z,): -0.0
single_ind(y < z,): 0.0
single_ind(x <= -y + z,): 0.0
single_ind(x <= y - z,): -0.0
single_ind(z < y,): 0.0
single_ind(7*x <= y,): -0.0
main_ind(4*x <= z, 3*x < y): -0.0
component_ind(4*x <= z,): 0.0
component_ind(3*x < y,): 0.0
single_ind(5*x <= y,): -0.0
single_ind(6*x <= z,): -0.0
main_ind(5*x <= 3*y, y < 3*x): 0.0
component_ind(5*x <= 3*y,): 0.0
component_ind(y < 3*x,): 1.0
main_ind(3*x <= y + z, y < 3*x): 0.0
component_ind(3*x <= y + z,): 0.0
component_ind(y < 3*x,): 1.0
main_ind(2*x <= y, y - z < x): 0.0
component_ind(2*x <= y,): 0.0
component_ind(y - z < x,): 1.0
single_ind(x <= -y + z,): -0.0
main_ind(3*x <= y + z, z < 2*x): 0.0
component_ind(3*x <= y + z,): 0.0
component_ind(z < 2*x,): 1.0
single_ind(z < 2*x,): 1.0
single_ind(3*x <= y - z,): 0.0
single_ind(x < y - z,): -0.0
single_ind(6*x <= y,): 0.0
main_ind(5*x <= z, 

In [ ]:
%run constants.ipynb
%run gen_h.ipynb
%run verification.ipynb

In [340]:
delta_dash(7, 332,334,334)

-0.03958261888431641

In [ ]:
delta_dash(12,332,334,334)

In [ ]:
delta_R(4,26,27,4)